In [1]:
from __future__ import division
import numpy as np
import csv
np.random.seed(0)

We now compare several algorithms for approximate matrix multiplication

Approximation algorithm: top-k sampling without scaling

In [2]:
def top_k(A,B,K):
    # calculate norms of the columns of A and rows of B
    a_col_norms = np.linalg.norm(A,axis=0)
    b_row_norms = np.linalg.norm(B,axis=1)
    
    # multiply both norms element-wise to and pick the indices of the top K column-row pairs
    norm_mult = np.multiply(a_col_norms,b_row_norms)
    top_k_indices = np.sort(np.argsort(norm_mult)[::-1][:K])
    
    # pick top-k column-row pairs to form new smaller matrices
    A_top_k_cols = A[:,top_k_indices]
    B_top_k_rows = B[top_k_indices,:]
       
    # multiply smaller matrices
    C_approx = np.dot(A_top_k_cols, B_top_k_rows)
    return C_approx

Approximation algorith: column-row sampling

In [3]:
''' A,B - input matrices
    K - number of column-row elements to sample
    with_replacement - True means sampling is done with replacement, False means sampling without replacement
    optimal_prob - True means sampling probability is proportional to |Ai|*|Bj|. False means random distribution
    scale - True means each column-row is scaled by 1/sqrt(K*pi) to ensure bias 0
'''
def column_row(A,B,K,with_replacement = True, optimal_prob = True, scale=True, debug=False):
    # calculate norms of the columns of A and rows of B
    a_col_norms = np.linalg.norm(A,axis=0)
    b_row_norms = np.linalg.norm(B,axis=1)
   
    # multiply both norms element-wise
    norm_mult = np.multiply(a_col_norms,b_row_norms)
    sum_norm_mult = np.sum(norm_mult)
    
    if optimal_prob == True and sum_norm_mult != 0:
        prob_dist = norm_mult/sum_norm_mult
    else:
        prob_dist = np.ones(A.shape[1])/A.shape[1] # uniform distributionwill be treated as uniform by np.random.choice 
    
    # scale input matrices according to probabilities.
    # For convenience we implement it by creating a diagonal matrix and multiplying (other implementations are possible).
    if scale == True:
        scale_matrix = np.diag(np.divide(1,np.sqrt(np.multiply(K,prob_dist))))
    else:
        scale_matrix = np.diag(np.ones(A.shape[1]))

    A_scaled = np.dot(A,scale_matrix)
    B_scaled = np.dot(scale_matrix,B)
    
    sample_indices = np.random.choice(A.shape[1], size=K, replace=with_replacement, p=prob_dist)
    
    # sample k column-row pairs to form new smaller matrices
    A_k_cols = A_scaled[:,sample_indices]
    B_k_rows = B_scaled[sample_indices,:]
       
    # multiply smaller matrices
    C_approx = np.dot(A_k_cols, B_k_rows)

    if debug == True:
        print ('a_col_norms is ' + str(a_col_norms))
        print ('b_row_norms is ' + str(b_row_norms))
        print ('norm_mult is ' + str(norm_mult))
        print ('sum_norm_mult is ' + str(sum_norm_mult))
        print ('prob_dist is ' + str(prob_dist))
        print ('scale matrix is ' + str(scale_matrix))
        print ('A_scaled is ' + str(A_scaled))
        print ('B_scaled is ' + str(B_scaled))
        print ('sample_indices are ' + str(sample_indices))
        print ('Frobenius error bound is '+ str(sum_norm_mult**2/K -  np.linalg.norm(A.dot(B)/K)))
        print ('A_k_cols is ')
        print (A_k_cols)
        print ('B_k_rows is ')
        print (B_k_rows)
        print ('C_approx is ')
        print (C_approx)
    
    return C_approx

Approximation algorithm - Bernoulli sampling

In [4]:
''' A,B - input matrices
    K - sampling parameter
    scale - True means each column-row is scaled by 1/sqrt(pi) to ensure bias 0
'''
def column_row_bern(A,B,K, scale=True, debug=False):
    # calculate norms of the columns of A and rows of B
    a_col_norms = np.linalg.norm(A,axis=0)
    b_row_norms = np.linalg.norm(B,axis=1)
   
    # multiply both norms element-wise
    norm_mult = np.multiply(a_col_norms,b_row_norms)
    sum_norm_mult = np.sum(norm_mult)
    
    if sum_norm_mult != 0:
        prob_dist = K*norm_mult/sum_norm_mult
    else:
        prob_dist = np.ones(A.shape[1]) 
    
    prob_dist = np.clip(prob_dist,0,1)
    
    # scale input matrices according to probabilities.
    # For convenience we implement it by creating a diagonal matrix and multiplying (other implementations are possible).
    if scale == True:
        scale_matrix = np.diag(np.divide(1,np.sqrt(prob_dist)))
    else:
        scale_matrix = np.diag(np.ones(A.shape[1]))

    A_scaled = np.dot(A,scale_matrix)
    B_scaled = np.dot(scale_matrix,B)
    
    bern = np.random.binomial(1, prob_dist)
    
    sample_indices = np.where(bern == 1)[0]
    
    # sample k column-row pairs to form new smaller matrices
    A_k_cols = A_scaled[:,sample_indices]
    B_k_rows = B_scaled[sample_indices,:]
       
    # multiply smaller matrices
    C_approx = np.dot(A_k_cols, B_k_rows)
  
    if debug == True:
        print ('a_col_norms is ' + str(a_col_norms))
        print ('b_row_norms is ' + str(b_row_norms))
        print ('norm_mult is ' + str(norm_mult))
        print ('sum_norm_mult is ' + str(sum_norm_mult))
        print ('prob_dist is ' + str(prob_dist))
        print ('scale matrix is ' + str(scale_matrix))
        print ('A_scaled is ' + str(A_scaled))
        print ('B_scaled is ' + str(B_scaled))
        print ('sample_indices are ' + str(sample_indices))
        print ('num sampled indices is ' + str(len(sample_indices)))
        print ('Frobenius error bound is '+ str(sum_norm_mult**2/K - np.sum(np.multiply(np.multiply(a_col_norms,a_col_norms),np.multiply(b_row_norms,b_row_norms)))))
        print ('A_k_cols is ')
        print (A_k_cols)
        print ('B_k_rows is ')
        print (B_k_rows)
        print ('C_approx is ')
        print (C_approx)
    
    return C_approx

Approximation algorithm - random walks (Cohen and Lewis, 1999)

In [5]:
''' A,B - input matrices
    S - number of samples (random walks per row)
    negative_entries - True means entries can be negative, False means not
'''
def random_walks(A,B,S,negative_entries = False, debug=False):
    walk_trace = np.zeros((A.shape[0],B.shape[1]))
    
    if negative_entries == False:
        # compute W for layer 2 nodes
        W2 = np.sum(B,axis=1)

        # compute W for layer  1 nodes
        W1 = np.dot(A,W2.T)

        # assign probabilities for layer 2
        P2_denominator = np.diag(1/W2)
        P2 = np.dot(P2_denominator, B)

        # assign probabilities for layer 1
        P1_nominator = np.dot(A, np.diag(W2))
        P1_denominator = np.diag(1/W1)
        P1 = np.dot(P1_denominator,P1_nominator)
    else:
        return
    
    # Scoring
    # loop over rows
    for node0 in range(A.shape[0]):
        # take S samples for each row
        for s in range(S):
            # travel from node0 to node1 according to probability distribution of row node0 in P1
            node1 = np.random.choice(P1.shape[1], size=1, p=P1[node0,:])[0]

            # travel from node1 to node2 according to probability distribution of row node1 in P2
            node2 = np.random.choice(P2.shape[1], size=1, p=P2[node1,:])[0]
            
            # update trace
            walk_trace[node0][node2] += 1
    
    # estimate product from trace
    C_approx = np.dot(np.diag(W1),walk_trace)/S
    return C_approx
    
    
    if debug == True:
        print ('W2 is ' + str(W2))
        print ('W1 is ' + str(W1))
        print ('P2_denominator is ' + str(P2_denominator))
        print ('P2 is ' + str(P2))
        print ('P1_denominator is ' + str(P1_denominator))
        print ('P1 is ' + str(P1))
        print ('C_approx is ' + str(C_approx))
        
    
    

Approximation algorithm - SVD

In [6]:
''' A,B - input matrices
    K - number of singular vectors to take from each matrix
    this function performs full SVD decomposition, which is more computationaly expensive than
    matrix multiplication itself. We will use it as a measure of maximal accuracy for other approximation algorithms 
'''
def svd_mul(A,B,K,debug=False):
    A_U, A_s, A_V = np.linalg.svd(A)
    B_U, B_s, B_V = np.linalg.svd(B)
    A_U_k = A_U[:,:K]
    A_s_k = A_s[:K]
    A_V_k = A_V[:K,:]
    B_U_k = B_U[:,:K]
    B_s_k = B_s[:K]
    B_V_k = B_V[:K,:]   
    
    A_s_k_A_V_k = np.dot(np.diag(A_s_k),A_V_k)
    B_U_k_B_s_k = np.dot(B_U_k, np.diag(B_s_k))
    A_s_k_A_V_k_B_U_k_B_s_k = np.dot(A_s_k_A_V_k, B_U_k_B_s_k)
    
    C_approx = np.dot(np.dot(A_U_k, A_s_k_A_V_k_B_U_k_B_s_k), B_V_k)
    return C_approx

Helper function to display approximation quality

In [7]:
def get_stats(A,B,approx):
    stats = {}
    
    # Calculate accurate multiplication result C=AB
    acc = np.dot(A,B)
    
    # Relative error of Frobenius norm   
    relative_frobenius_error = np.abs(np.linalg.norm(acc, ord='fro')-np.linalg.norm(approx, ord='fro'))/np.linalg.norm(acc, ord='fro')

    # Relative error of spectal norm
    relative_spectral_error = np.abs(np.linalg.norm(acc, ord=2)-np.linalg.norm(approx, ord=2))/np.linalg.norm(acc, ord=2)
      
    # Frobenius norm of the error matrix (acc-approx)
    error_matrix_frobenius = np.linalg.norm(acc-approx, ord='fro')
    
    # Normalized Frobenius error F(acc-approx)/(F(A)F(B))
    normalized_frobenius_error= np.linalg.norm(acc-approx, ord='fro')/(np.linalg.norm(A, ord='fro')*np.linalg.norm(B, ord='fro'))
    
    # Spectral norm of the error matrix
    error_matrix_spectral = np.linalg.norm(acc-approx, ord=2)
        
    # Average per-element error:
    per_element_error_avg = np.mean(np.abs(acc-approx))
        
    # Std of the per-element error:
    per_element_error_std = np.std(acc-approx)
    
    stats['relative_frobenius_error'] = relative_frobenius_error
    stats['relative_spectral_error'] = relative_spectral_error
    stats['error_matrix_frobenius'] = error_matrix_frobenius
    stats['normalized_frobenius_error'] = normalized_frobenius_error
    stats['error_matrix_spectral'] = error_matrix_spectral
    stats['per_element_error_avg'] = per_element_error_avg
    stats['per_element_error_std'] = per_element_error_std
    
    return stats
    

In [8]:
def print_stats(stats, name):

    print ('Statistics for approximation method ' + name)
    print ('Relative Frobenius error is: ' + str(stats['relative_frobenius_error']))
    print ('Relative Spectral error is: ' + str(stats['relative_spectral_error']))
    print ('Frobenius norm of error matrix: ' + str(stats['error_matrix_frobenius']))
    print ('Frobenius norm of error matrix normalized: ' + str(stats['normalized_frobenius_error']))
    print ('Spectral norm of error matrix: ' + str(stats['error_matrix_spectral']))
    print ('Average per-element error: ' + str(stats['per_element_error_avg']))
    print ('Standard deviation of the per-element error: ' + str(stats['per_element_error_std']))

First experiment - 100x100 matrix multiplication

In [16]:
# initialize 100x100 random matrices (~N(0,1)
M = 100
K = 100
N = 100

# Number of experiments per data point to average on
repeat_experiments = 1000

algorithms = ['top_k', 
              #'colrow_uni_rep',
              'colrow_opt_rep',
              #'colrow_uni_norep',
              #'colrow_opt_norep',
              #'colrow_uni_rep_noscale',
              #'colrow_opt_rep_noscale',
              #'colrow_uni_norep_noscale',
              #'colrow_opt_norep_noscale',
              'colrow_bern_scale']#,
              #'colrow_bern_noscale']
              #'svd_mul']

stat_names = ['relative_frobenius_error',
              'relative_spectral_error',
              'error_matrix_frobenius',
              'normalized_frobenius_error',
              'error_matrix_spectral',
              'per_element_error_avg',
              'per_element_error_std']

writers = {}
files = []
C_approx = {}

# Create one output csv file per stat
for i in range(len(stat_names)):
    files.append(open(stat_names[i] + '.csv', 'w', newline=''))
    writers[stat_names[i]] = csv.writer(files[i])
    writers[stat_names[i]].writerow([stat_names[i]])
    if stat_names[i] != 'normalized_frobenius_error':
        writers[stat_names[i]].writerow(['k'] + algorithms)
    else:
        writers[stat_names[i]].writerow(['k'] + algorithms + ['normalized frobenius error upper bound'])

for k in range(1,K+1,1):    
    print ('k is ' + str(k))
    C_stats_sum = {}
    for i in range(len(stat_names)):
        C_stats_sum[stat_names[i]] = {}
        for j in range(len(algorithms)):
            C_stats_sum[stat_names[i]][algorithms[j]] = 0.0
    
    for n in range(repeat_experiments):       
    
        # Sample input matrices
        #A = np.random.normal(loc=np.random.randint(-1,1), scale=1, size=(M,K))
        #A = np.zeros([M,K])
        #A = np.random.zipf(a=1e8, size=(M,K))
        A = np.random.normal(loc=1, scale=1, size=(M,K))
        #B = np.random.normal(loc=np.random.randint(-1,1), scale=1, size=(K,N))
        B = np.random.normal(loc=1, scale=1, size=(K,N))
        #B = np.zeros([K,N])
        #for i in range(K):
        #    l = np.random.randint(-(i+1),i+1)
        #    B[i,:] = np.random.normal(loc=l, scale=1, size=(1,N))
        #    A[:,i] = np.random.normal(loc=l, scale=1, size=(1,M))
        
        #B = np.random.zipf(a=1e8, size=(K,N))
        #B = np.random.normal(loc=1, scale=1, size=(K,N))

        
        
        # Calculate the product AB using different approximation algorithms
        C_approx['top_k'] = top_k(A,B,k)
        #C_approx['colrow_uni_rep'] = column_row(A,B,k,with_replacement=True, optimal_prob=False, scale=True)
        C_approx['colrow_opt_rep'] = column_row(A,B,k,with_replacement=True, optimal_prob=True, scale=True)
        #C_approx['colrow_uni_norep'] = column_row(A,B,k,with_replacement=False, optimal_prob=False, scale=True)
        #C_approx['colrow_opt_norep'] = column_row(A,B,k,with_replacement=False, optimal_prob=True, scale=True)
        #C_approx['colrow_uni_rep_noscale'] = column_row(A,B,k,with_replacement=True, optimal_prob=False, scale=False)
        #C_approx['colrow_opt_rep_noscale'] = column_row(A,B,k,with_replacement=True, optimal_prob=True, scale=False)
        #C_approx['colrow_uni_norep_noscale'] = column_row(A,B,k,with_replacement=False, optimal_prob=False, scale=False)
        #C_approx['colrow_opt_norep_noscale'] = column_row(A,B,k,with_replacement=False, optimal_prob=True, scale=False)
        C_approx['colrow_bern_scale'] = column_row_bern(A,B,k, scale=True)
        #C_approx['colrow_bern_noscale'] = column_row_bern(A,B,k, scale=False)
        #C_approx['svd_mul'] = svd_mul(A,B,k)
        
        # Get statistics for each algorithm
        C_stats_current = {}
        for j in range(len(algorithms)):
            C_stats_current[algorithms[j]] = get_stats(A,B, C_approx[algorithms[j]])
            for i in range(len(stat_names)):
                C_stats_sum[stat_names[i]][algorithms[j]] += C_stats_current[algorithms[j]][stat_names[i]]
        
    
    # Output format:
    # a separate csv per statistic
    # for each csv - 
    #     the columns are the same statistic for different algorithms
    #     the rows are different values of k sampled 
    for i in range(len(stat_names)):
        stats = [k]
        for j in range(len(algorithms)): 
            stats.append(C_stats_sum[stat_names[i]][algorithms[j]]/repeat_experiments)
        if stat_names[i] != 'normalized_frobenius_error':
            writers[stat_names[i]].writerow(stats)
        else:
            writers[stat_names[i]].writerow(stats + [1/np.sqrt(k)])

    
# Close stat files
for i in range(len(stat_names)):
    files[i].close()

k is 1
k is 2
k is 3
k is 4
k is 5
k is 6
k is 7
k is 8
k is 9
k is 10
k is 11
k is 12
k is 13
k is 14
k is 15
k is 16
k is 17
k is 18
k is 19
k is 20
k is 21
k is 22
k is 23
k is 24
k is 25
k is 26
k is 27
k is 28
k is 29
k is 30
k is 31
k is 32
k is 33
k is 34
k is 35
k is 36
k is 37
k is 38
k is 39
k is 40
k is 41
k is 42
k is 43
k is 44
k is 45
k is 46
k is 47
k is 48
k is 49
k is 50
k is 51
k is 52
k is 53
k is 54
k is 55
k is 56
k is 57
k is 58
k is 59
k is 60
k is 61
k is 62
k is 63
k is 64
k is 65
k is 66
k is 67
k is 68
k is 69
k is 70
k is 71
k is 72
k is 73
k is 74
k is 75
k is 76
k is 77
k is 78
k is 79
k is 80
k is 81
k is 82
k is 83
k is 84
k is 85
k is 86
k is 87
k is 88
k is 89
k is 90
k is 91
k is 92
k is 93
k is 94
k is 95
k is 96
k is 97
k is 98
k is 99
k is 100


In [ ]:
D = np.array([[1,2,3],[4,5,6]])
E = np.array([[7,8],[9,10],[11,12]])
k=2
print D
print E
print np.dot(D,E)

In [22]:
random_walks(D,E,13)

array([[  56.30769231,   65.69230769],
       [ 112.69230769,  180.30769231]])

In [16]:
svd_mul(D,E,1)

array([[  58.17058972,   64.43900704],
       [ 138.88714117,  153.85351103]])

In [79]:
A = np.random.normal(loc=0, scale=1, size=(5,5))
B = np.random.normal(loc=0, scale=1, size=(5,5))
column_row_bern(A,B,2, scale=True, debug=True)-A.dot(B)


#random_walks(A,B,1000)


a_col_norms is [2.11667059 1.94337924 1.82620943 2.35940971 1.48550002]
b_row_norms is [1.35103635 2.16050556 2.36527216 3.28954819 1.16040812]
norm_mult is [2.8596989  4.19868166 4.31948233 7.76139195 1.72378629]
sum_norm_mult is 20.863041116407594
prob_dist is [0.27414018 0.40249949 0.41407984 0.74403266 0.16524784]
scale matrix is [[1.90991331 0.         0.         0.         0.        ]
 [0.         1.57622182 0.         0.         0.        ]
 [0.         0.         1.55402485 0.         0.        ]
 [0.         0.         0.         1.1593218  0.        ]
 [0.         0.         0.         0.         2.45998296]]
A_scaled is [[-0.43341211  2.76578687 -0.7452519  -0.90286895  1.97687315]
 [-2.30714529  0.76485705  0.06841987 -2.49133003 -0.91792323]
 [ 0.612707   -0.98997047  0.81475097 -0.574065    1.54083665]
 [ 1.12231855  0.18524605  2.60254433 -0.02447616 -2.29819639]
 [ 3.03270574 -0.36637881  0.23866989 -0.36041828  0.97336954]]
B_scaled is [[-2.06867715  0.32470341 -0.2160

array([[-1.53042751, -2.07972341,  3.26291153, -4.39453807, -0.56630209],
       [-1.83680571, -1.0105038 ,  2.33457578, -2.89365602,  0.75972393],
       [-0.417174  ,  0.76652379, -0.11786664,  1.20005904, -0.76557205],
       [-1.04086437, -0.28384309, -0.55304795, -1.64340119, -1.98243847],
       [ 1.35697524,  0.03637697,  0.32031548,  1.59567068, -0.16347814]])

In [56]:
svd_mul(A,B,10)

array([[ -0.12353307,   2.70736565,   0.08641966, ...,   1.23592005,
         -6.78048297,   0.56506958],
       [  2.71544266,   0.08661535,   0.12344578, ...,  -8.42435018,
          4.88709719,  -0.16644529],
       [ -9.82749755,   9.39555596,  14.12118491, ...,  -4.63854117,
         -9.72714385,  10.20529936],
       ..., 
       [  0.39364931,   5.22158707,  -0.96962055, ..., -10.40712015,
          9.66697407,  -0.64169166],
       [  3.36894029,  -4.58071512,  -3.42604357, ...,  -1.31869436,
          5.15108826,  -2.62539348],
       [  1.93828184,  -0.44931835,  -1.28170357, ...,  -8.33528075,
         10.81260351,  -0.60708691]])

In [43]:
np.dot(A,B)

array([[ -6.43568934,  -3.90850717,  -4.50532006, ..., -12.67101179,
        -10.15682965, -13.75763405],
       [  9.7159455 ,   5.77210303,  -6.70278586, ..., -14.93998827,
         10.07291841,  -9.7994708 ],
       [  5.50987195,  15.09380985,  21.62866239, ...,  -6.55928461,
        -11.92391737,   4.81192288],
       ..., 
       [-14.33451067,  -5.04033338,  11.80525566, ..., -12.38111982,
         -6.37253943,   4.20435141],
       [ -2.22791811,   0.33741418,   3.82464388, ...,  -0.50160204,
         26.72042787, -11.79607345],
       [  2.04571995,   1.14928371,  -0.52280088, ..., -10.55945816,
         -2.95092781,  13.93082587]])

In [55]:
top_k(A,B,10)

array([[-7.83325374,  1.81662122,  1.10048855, ..., -3.89241902,
         1.0590615 , -0.80584154],
       [-1.45229436,  4.47517318, -2.77303134, ...,  0.54945197,
         2.17507427, -1.27126498],
       [ 1.93681846,  4.02029345,  2.38006873, ..., -3.93959194,
        -1.10964247, -4.93127999],
       ..., 
       [ 3.54270866, -8.78806713, -3.66210396, ...,  5.23597104,
         0.55799719, -0.15416489],
       [-6.07363412,  5.35158953,  0.55807345, ...,  1.25972357,
        -0.28438256, -5.133833  ],
       [ 9.08320794, -4.20921349, -2.06990813, ..., -0.70497197,
        -0.91626798,  5.23245363]])

In [44]:
A = np.array([[-0.56948143,0.6234533,0.39866474,0.21468586,-0.8736436 ],
 [ 0.83005655,  0.09830075, -0.03885731, -0.2037122,   1.9092574, ],
 [-1.7054096,  -0.10213567,  0.15305512, -1.164082,   -1.1722519 ],
 [-0.4288956,   1.1507404,  -1.7422528,  -0.02039167, -0.00957074],
 [ 1.8480316,  -0.8813653,  -0.07361737,  0.85047555,  0.43315265]]
)
B = np.array([[ 0.2528034,   0.85932934,  0.7415126,  -0.7723672,   1.1472509 ],
 [-0.7447469,  -0.50370145,  0.18461666,  0.16899914,  0.93716013],
 [-0.5356013,  -1.5470285,  -2.0807953,   0.4024434,   0.8996739 ],
 [-1.7614233,  -1.2215439,  -0.75340575, -0.23610812, -1.4844245 ],
 [ 0.37762547, -0.40104532, -0.31718078,  0.71741456, -0.9783565 ]])
A = np.random.normal(size=(50,50))
B = np.random.zipf(a=2, size=(50,50))


In [45]:
print(A.dot(B))

[[  11.08711404  -54.09708129   35.04564521 ...,  -64.69599955
   -30.18062722  -32.83347395]
 [  -3.65045329  275.15997442    7.58957963 ..., -175.13654913
    14.61493058  -12.96219937]
 [ -21.81520671  409.07492208  -36.71183062 ...,    1.10830785
   -30.60327048  -21.77099869]
 ..., 
 [  20.6222693   -17.56464522   34.83065905 ...,  126.64939202   17.6626778
    13.4089341 ]
 [-117.1839508    -8.82787764  -15.5182248  ...,  -31.30414425
     2.61513924   20.00294063]
 [ -60.88094067 -201.32990652  -36.91815021 ...,  -54.50376384
    16.67376514  -15.59686276]]


In [75]:
print(column_row(A,B,25,with_replacement=True, optimal_prob=True, scale=False))

[[  47.38387061  -15.67847164  -15.28636685 ...,    3.00901655
    -1.14483605    9.06508816]
 [-154.36400622  -39.82783016  -35.14531024 ...,  -32.93814493
   -31.91303924  -54.88639593]
 [ -10.7142406    10.16820778   -5.87699611 ...,   -0.89513589
    -0.60178752   -3.7310086 ]
 ..., 
 [  24.44056688   46.8047143    17.79101094 ...,    9.90229623
    12.96267789   18.77174239]
 [  26.61743539  -45.33496999  -19.80077321 ...,    1.59527248
    -5.33122068    4.11783945]
 [ -63.45978797  -79.972869    -30.41461971 ...,  -13.13251341
   -19.02233942  -17.72792278]]


In [47]:
print(np.linalg.norm(column_row(A,B,5,with_replacement=False, optimal_prob=True, scale=True)-A.dot(B)))

89857.6214422


In [48]:
print(top_k(A,B,25))

[[  1.03527263e+01  -3.41643706e+01   1.21056433e+01 ...,   6.20017050e+01
   -7.85729402e+00  -1.51572029e+01]
 [  1.90507469e+00   2.78138373e+02   1.74872795e-01 ...,  -6.36404616e+01
    6.26443194e+00  -1.73418464e+01]
 [ -1.85158693e+01   4.36089868e+02  -3.36040592e+01 ...,   4.66281294e+01
    1.10722363e+01  -5.07384593e+00]
 ..., 
 [  1.61779260e+01  -3.30297336e+01   3.74283192e+01 ...,  -1.60047771e+01
    8.41049612e+00   1.43385627e+01]
 [ -1.18756131e+02  -1.13690487e+01  -4.40885771e+01 ...,  -5.13886032e+00
   -7.21229172e+00  -1.70976270e+01]
 [ -4.97545295e+01  -2.15840289e+02  -1.54925629e+01 ...,   2.68197579e+01
   -2.11246927e+01  -1.21211718e+01]]
